# HERFD co-pilot

In [2]:
%matplotlib widget
import os
import matplotlib.pyplot as plt
import numpy as np
from silx.io.specfile import SpecFile
from scipy.signal import savgol_filter

BASE_PATH = r'/data/bm20/inhouse/data_TOP/20221004_Vejar_U_Pu'

def average(
    spec_filename,
    scan_numbers,
    export_base_filename = None
):
    """
    Average (and export to file if desired) multiple scans
    """
    sf = SpecFile(os.path.join(BASE_PATH, spec_filename))
    
    plt.figure()
    
    # Determine scan shape
    energy = sf[scan_numbers[0] - 1].data_column_by_name('energy')
    data     = np.empty( (len(scan_numbers), len(energy)) )
    data[:]  = np.nan
    
    for idx, scan_no in enumerate(scan_numbers):
        
        scan_no = scan_no - 1

        x = sf[scan_no].data_column_by_name('energy')
        y = sf[scan_no].data_column_by_name('ketek')
        i0 = sf[scan_no].data_column_by_name('i0s')
        
        idx0, idx1 = np.argmin(np.abs(energy - x[0])), np.argmin(np.abs(energy - x[-1]))
        
        if not np.all(np.round(energy[idx0:idx1 + 1], 2) == np.round(x, 2)):
            print("Warning! You seem to average scans with different energy axes.")

        data[idx, idx0:idx1 + 1] = y
        
        
            
        plt.plot(x, y, lw = 0.5)
    
    data = np.nansum(data, axis = 0)
    error = np.sqrt(data)
    
    # Plot the averaged data and error
    plt.title(export_base_filename)
    plt.plot(energy, data, 'k.-', lw = 2, label = 'averaged')
    plt.fill_between(energy, data - error/2, data + error/2, alpha = 0.25, color = 'k', label = 'Poisson error $\sqrt{N}$')
    plt.xlabel('Energy')
    plt.ylabel('Intensity (counts)')
    
    data_norm = data / np.nanmax(data) 
    data_savgol = savgol_filter(data, window_length = 3, polyorder = 1)
    
    # Plot the smoothed data
    plt.plot(energy, data_savgol, 'r', lw = 1, label = 'SG-filtered')
    plt.legend()
    
    data_savgol = data_savgol / np.nanmax(data_savgol)
    
    # If a basename was given, export 
    # - the average, 
    # - average normalized and 
    # - avarge normalized smoothed data
    
    if not export_base_filename is None:
        
        filename = export_base_filename + '.txt'  
        filename_n1 = export_base_filename + '_normalized.txt'
        filename_sbg = export_base_filename + '_normalized_sgb.txt'
        
        
        print(f'Info: Saving result to files:')
        print(f'Average: {filename}:')
        print(f'Normalized: {filename_n1}:')
        print(f'Smoothed: {filename_sbg}:')
        
        np.savetxt(os.path.join(BASE_PATH, filename), np.vstack([energy, data]).T)
        np.savetxt(os.path.join(BASE_PATH, filename_n1), np.vstack([energy, data_norm]).T)
        np.savetxt(os.path.join(BASE_PATH, filename_sbg), np.vstack([energy, data_savgol]).T)
 
    
    
def compare(
    filenames,
):
    """
    Plot several scans from different samples for easier comparison
    """
        
    plt.figure()
    for filename in filenames:
        x, y = np.loadtxt(os.path.join(BASE_PATH, filename)).T
        plt.plot(x, y, '.-', label = filename)
        
    plt.legend()

# Average datasets

In [7]:
average(
    spec_filename = 'Pu21_21', 
    scan_numbers = [4, 5, 6, 7], 
    export_base_filename = 'Pu_M4_edge_HERFD/Pu21_21_florian_'
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Info: Saving result to files:
Average: Pu_M4_edge_HERFD/Pu21_21_florian_.txt:
Normalized: Pu_M4_edge_HERFD/Pu21_21_florian__normalized.txt:
Smoothed: Pu_M4_edge_HERFD/Pu21_21_florian__normalized_sgb.txt:


# Compare datasets

In [8]:
compare([
    'Pu_M4_edge_HERFD/Pu21_21_florian_normalized.txt',
    # ...
])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …